create empty mne-python raw object

In [1]:
import numpy as np
import mne

# Create a simple RawArray
sfreq = 250  # Sampling frequency
ch_names = [f'EEG{d}' for d in range(1,75)]
ch_types = ["eeg"] * 74
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

data = np.random.randn(74, 747750)  # 2 channels, 1000 samples
raw = mne.io.RawArray(data, info)

print(isinstance(raw, mne.io.Raw))  # True
print(type(raw))  # <class 'mne.io.array.array.RawArray'>

Creating RawArray with float64 data, n_channels=74, n_times=747750
    Range : 0 ... 747749 =      0.000 ...  2990.996 secs
Ready.
False
<class 'mne.io.array.array.RawArray'>


braindecode call __getitem__ of mne.base.Raw, which then calls _getitem which calls _read_segment of BaseRaw. mne uses _read_segment to read a specific range of the file. We want to test whether S3 file via fsspec can be integrated
It calls _read_segments_file of the BaseRaw class. Any subclass must implement this method. EEGLAB calls fiff reader function: mne/_fiff/utils.py#L200

In [1]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2
import sys 
sys.path.append('..')
from eegdash import EEGDash
from eegdash.data_utils import RawEEGDash
from braindecode.datasets import BaseDataset, BaseConcatDataset

In [ ]:
signalstore = EEGDash(
    is_public=False,
)

In [3]:
record = signalstore.find({'dataset': 'ds002718', 'subject': '002'})[0]
sfreq = record['sampling_frequency']
nchans = record['nchans']
ntimes = record['ntimes']
ch_names = record['channel_names']
ch_types = record['channel_types']
s3_path = signalstore.get_s3path(record)
print(s3_path)


s3://openneuro.org/ds002718/sub-002/eeg/sub-002_task-FaceRecognition_eeg.set


In [4]:
query = {'dataset': 'ds005511', 'subject': 'NDARUF236HM7'}
datasets = []
for record in signalstore.find(query):
    sfreq = record['sampling_frequency']
    nchans = record['nchans']
    ntimes = record['ntimes']
    ch_names = record['channel_names']
    ch_types = record['channel_types']
    s3_path = signalstore.get_s3path(record)
    datasets.append(BaseDataset(RawEEGDash(s3_path, {'sfreq': sfreq, 'nchans': nchans, 'n_times': ntimes, 'ch_types': ch_types, 'ch_names': ch_names}, preload=False)))

integrate with braindecode

In [5]:
eegdash_braindecode = BaseConcatDataset(datasets)

In [5]:
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows)
windows_ds = create_fixed_length_windows(eegdash_braindecode, start_offset_samples=0, stop_offset_samples=None,
        window_size_samples=1000,
        window_stride_samples=1000, drop_last_window=True,
        preload=False)

In [6]:
import torch
torch.tensor([windows_ds[w][0] for w in range(len(windows_ds))]).shape

s3file s3://openneuro.org/ds005511/sub-NDARUF236HM7/eeg/sub-NDARUF236HM7_task-RestingState_eeg.set
filecache ./.eegdash_cache/sub-NDARUF236HM7_task-RestingState_eeg.set
s3file s3://openneuro.org/ds005511/sub-NDARUF236HM7/eeg/sub-NDARUF236HM7_task-symbolSearch_eeg.set
filecache ./.eegdash_cache/sub-NDARUF236HM7_task-symbolSearch_eeg.set
s3file s3://openneuro.org/ds005511/sub-NDARUF236HM7/eeg/sub-NDARUF236HM7_task-seqLearning6target_eeg.set
filecache ./.eegdash_cache/sub-NDARUF236HM7_task-seqLearning6target_eeg.set
s3file s3://openneuro.org/ds005511/sub-NDARUF236HM7/eeg/sub-NDARUF236HM7_task-surroundSupp_run-1_eeg.set
filecache ./.eegdash_cache/sub-NDARUF236HM7_task-surroundSupp_run-1_eeg.set


/tmp/ipykernel_3340652/2020391110.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  torch.tensor([windows_ds[w][0] for w in range(len(windows_ds))]).shape


torch.Size([1793, 129, 1000])